In [24]:
# Converting a Lambda function into runnable. Any function that is converted to a runnable can them be invoked and chained

In [34]:
from langchain_core.runnables import RunnableLambda, chain

In [2]:
find_sum = lambda x: sum(x)

In [4]:
find_sum([1,2,3])

6

In [6]:
find_square = lambda x: x**2

In [8]:
find_square(8)

64

In [12]:
runnable_sum = RunnableLambda(lambda x : sum(x))

In [14]:
runnable_sum.invoke([1,2,3])

6

In [16]:
runnable_square = RunnableLambda(lambda x: x**2)
runnable_square.invoke(8)

64

In [18]:
chain = runnable_sum | runnable_square

In [20]:
chain.invoke([1,2,3])

36

In [22]:
chain.get_graph().print_ascii()

+-------------+  
| LambdaInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| LambdaOutput | 
+--------------+ 


In [ ]:
# Langchain decorators

In [26]:
def findsum(x):
    return sum(x)

In [28]:
def findsquare(x):
    return(x**2)

In [30]:
chain1 = RunnableLambda(find_sum) | RunnableLambda(find_square)

In [32]:
chain1.invoke([1,2,3])

36

In [36]:
@chain # This decorator wraps the two function as a Runnable lambda
def runnablesum(x):
    return sum(x)
@chain
def runnablesquare(x):
    return(x**2)    

In [38]:
type(runnable_sum)

langchain_core.runnables.base.RunnableLambda

In [40]:
chain2 = runnable_sum | runnable_square

In [42]:
chain2.invoke([1,2,3])

36

In [ ]:
# When you use @chain decorator, dont have a variable with the name chain

In [44]:
%load_ext dotenv
%dotenv

In [96]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from langchain.memory import ConversationSummaryMemory
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain_core.runnables import chain

In [98]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI.
The AI is talkative and provids lots of specific details from its context.
If the AI does not know the answer to a question, it trutfully says that it does not know.

Current Conversation:
{message_log}

Human:
{question}

AI:
'''

In [54]:
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [100]:
chat = ChatOpenAI ( model='gpt-4', 
                   seed=365 , 
                   temperature=0, 
                   max_completion_tokens=100)

In [58]:
chain = prompt_template | chat | StrOutputParser()

In [60]:
chain.invoke({'message_log':'', 
             'question':'Can you give me an interesting fact i probably didnt know about'})

'Sure, did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. And interestingly, when an octopus swims, the heart that delivers blood to the body stops beating, which is why octopuses prefer to crawl rather than swim, swimming is exhausting for them.'

In [62]:
chain.invoke({'message_log':'Can you give me an interesting fact i probably didnt know about.Sure, did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. And interestingly, when an octopus swims, the heart that delivers blood to the body stops beating, which is why octopuses prefer to crawl rather than swim, swimming is exhausting for them', 
             'question':'Can you elaborate a bit more'})

'Absolutely! Octopuses are fascinating creatures. As I mentioned, they have three hearts. The two hearts that pump blood to the gills are called branchial hearts, and the third one that pumps blood to the rest of the body is called the systemic heart. \n\nWhen an octopus is resting or moving slowly, all three hearts are working. But when it swims, the systemic heart, which supplies blood to the body, stops beating. This is because swimming requires more energy and oxygen, so'

In [ ]:
# We should not be copy pasting response into message log. Thas the job of a memory object

In [102]:
chat_memory = ConversationSummaryMemory( llm = ChatOpenAI(), memory_key='message_log')

In [68]:
chat_memory.load_memory_variables({})

{'message_log': ''}

In [72]:
# We need to hook up the message_log variable with the input variable of the same name
RunnablePassthrough().invoke({'question':'Can you give me an interesting fact i probably didnt know about?'})

{'question': 'Can you give me an interesting fact i probably didnt know about?'}

In [ ]:
# In order to assign the memory to the this invoke method

In [76]:
RunnablePassthrough.assign(message_log = chat_memory.load_memory_variables).invoke({'question':'Can you give me an interesting fact i probably didnt know about?'})

{'question': 'Can you give me an interesting fact i probably didnt know about?',
 'message_log': {'message_log': ''}}

In [ ]:
# itemgetter method to fetch items from object in list tupels dicts etc

In [80]:
itemgetter('message_log')({'message_log':''})

''

In [ ]:
# Make this a runnable to invoke it, use itemgetter is called as part of a chain

In [90]:
RunnableLambda(itemgetter('message_log')).invoke({'message_log':''})

''

In [94]:
RunnablePassthrough.assign(message_log = 
                            RunnableLambda(chat_memory.load_memory_variables)|
                            RunnableLambda(itemgetter('message_log'))
                          ).invoke(
            {'question':'Can you give me an interesting fact i probably didnt know about?'})

{'question': 'Can you give me an interesting fact i probably didnt know about?',
 'message_log': ''}

In [ ]:
## The following code brings it all together

In [140]:
question = 'Can you give me an interesting fact i probably didnt know about?'

In [142]:
RunnablePassthrough().invoke({'question':question})

{'question': 'Can you give me an interesting fact i probably didnt know about?'}

In [144]:
# Now use the assign method to expand the dictionary to add the message_log key value pair
dictionary_output = RunnablePassthrough.assign(message_log = 
                            RunnableLambda(chat_memory.load_memory_variables)|
                            RunnableLambda(itemgetter('message_log'))
                          ).invoke({'question':question})

In [146]:
prompt_value_output = prompt_template.invoke(dictionary_output)

In [148]:
ai_message_output = chat.invoke(prompt_value_output)

In [149]:
response = StrOutputParser().invoke(ai_message_output)

In [150]:
chat_memory.save_context( inputs = {'input':question},
                         outputs = {'output':response})

In [151]:
chat_memory.load_memory_variables({})

{'message_log': 'The human asks the AI for an interesting fact they may not know about. The AI shares that octopuses have three hearts and explains why they prefer crawling to swimming. The human then asks for another fact, and the AI mentions that honey never spoils. Lastly, the AI shares that the Eiffel Tower can be taller during the summer due to the expansion of iron on hot days.\n'}

In [152]:
chain1= (
    RunnablePassthrough.assign(message_log = 
                            RunnableLambda(chat_memory.load_memory_variables)|
                            itemgetter('message_log'))
       | prompt_template
       | chat
       | StrOutputParser()                           
)
    

In [153]:
response = chain1.invoke({'question':question})
chat_memory.save_context( inputs = {'input':question},
                         outputs = {'output':response})

In [154]:
response

'Sure, did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. Interestingly, when an octopus swims, the heart that delivers blood to the body stops beating, which is why they prefer to crawl rather than swim, as swimming exhausts them.'

In [ ]:
# Creating a function

In [182]:
@chain
def memory_chain(question):
    chain1= (
    RunnablePassthrough.assign(message_log = 
                            RunnableLambda(chat_memory.load_memory_variables)|
                            itemgetter('message_log'))
       | prompt_template
       | chat
       | StrOutputParser()                           
    )

    #chain1.get_graph().print_ascii()
    
    response = chain1.invoke({'question':question})
    chat_memory.save_context( inputs = {'input':question},
                         outputs = {'output':response})    
    return response


In [184]:
memory_chain.invoke("Can you give me an interesting fact i probably didnt know about?")

'Sure, did you know that a day on Venus is longer than a year on Venus? This is because Venus has an extremely slow rotation on its axis. It takes about 243 Earth days for Venus to complete one rotation, but it only takes around 225 Earth days for Venus to complete one orbit around the Sun. So, a day on Venus (the period of rotation) is longer than its year (the period of orbit).'

In [185]:
memory_chain.invoke("Can you elaborate a bit more on this fact?")

"Of course, I'd be happy to elaborate on the fact about Venus. Venus has a very slow rotation on its axis. It takes about 243 Earth days for Venus to complete one rotation on its axis, which is what we consider a day on Venus. However, Venus orbits the Sun much faster than it rotates. It takes Venus about 225 Earth days to complete one orbit around the Sun, which is what we consider a year on Venus. So, a day on Venus (the time it"